### Машинное Обучения

## Домашнее задание №2 - Дерево решений

**Общая информация**

**Срок сдачи:** 23 апреля 2024, 23:59   
**Штраф за опоздание:** -2 балла за каждые 2 дня опоздания

Решений залить в свой github репозиторий.

Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс MyDecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict . (Но название класса не менять)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import entropy
from sklearn import tree


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        self.feature_importances_ = None
        self.criterion = criterion
        self.tree = {}


    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]


    def get_impurity(self, feature, y):
        """
        Функция вычисляет минимальное impurity для признака,
        используется адаптированный для классификация
        эффективный алгоритм из ноутбука с занятия
        Принимает признак и целевые переменные
        """
        best_impurity = float('inf')
        best_thr = None

        # сортировка значений признака и классов
        idx = np.argsort(feature)
        y_sorted, x_sorted = y[idx], feature[idx]

        # всего объектов
        total = len(y_sorted)
        # количество объектов каждого класса
        class_counts = np.bincount(y_sorted, minlength=np.max(y_sorted) + 1)
        # количество классов
        num_classes = len(class_counts)

        # подготовка кумулятивных счетчиков для классов
        # np.eye создает матрицу, где каждая строка соответствует one-hot кодировке класса для каждого объекта.
        # кумулятивная сумма классов слева от каждого возможного разбиения (не включая последнее значение для избежания пустого разбиения)
        left_counts = np.cumsum(np.eye(num_classes)[y_sorted], axis=0)[:-1]
        # оставшиеся счетчики классов справа для каждого разбиения
        right_counts = class_counts - left_counts

        # размеры левой и правой части для каждого возможного разбиения
        # индексы, которые будут использоваться как границы разбиения (от 1 до total-1).
        sizes = np.arange(1, total)
        # количество элементов слева и справа от каждого возможного разбиения
        left_sizes = sizes
        right_sizes = total - sizes


        if self.criterion == 'gini':
            left_impurity = 1 - np.sum((left_counts / left_sizes[:, None]) ** 2, axis=1)
            right_impurity = 1 - np.sum((right_counts / right_sizes[:, None]) ** 2, axis=1)
        elif self.criterion == 'entropy':
            left_prob = left_counts / left_sizes[:, None]
            right_prob = right_counts / right_sizes[:, None]
            left_impurity = -np.sum(left_prob * np.log2(left_prob, where=left_prob > 0), axis=1)
            right_impurity = -np.sum(right_prob * np.log2(right_prob, where=right_prob > 0), axis=1)
        elif self.criterion == 'misclassification':
            left_impurity = 1 - np.max(left_counts / left_sizes[:, None], axis=1)
            right_impurity = 1 - np.max(right_counts / right_sizes[:, None], axis=1)
        else:
            raise ValueError("Wrong criterion")

        impurity = (left_impurity * sizes + right_impurity * right_sizes) / total

        #  пропускает одинаковые значения признака
        mask = np.append(True, x_sorted[1:] != x_sorted[:-1])

        # применение маски для фильтрации impurity
        filtered_impurity = np.where(mask[1:], impurity, float('inf'))

        # нахождение минимального значения impurity
        min_idx = np.argmin(filtered_impurity)
        best_impurity = filtered_impurity[min_idx]
        best_thr = (x_sorted[min_idx + 1] + x_sorted[min_idx]) / 2

        return best_impurity, best_thr

    def __find_threshold(self, X, y):
        """
        функция, находящая лучший разделитель и
        """
        best_feature = None
        best_threshold = None
        best_impurity = float('inf')

        for j in range(X.shape[1]):
            impurity, threshold = self.get_impurity(X[:, j], y)
            if impurity < best_impurity:
                best_impurity = impurity
                best_feature = j
                best_threshold = threshold

        return best_feature, best_threshold

    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        if depth == self.max_depth or x.shape[0] < self.min_samples_split or np.unique(y).size == 1:
            # создаем лист, если мы достигли максимальной глубины, или количество объектов меньше,
            # чем наш гиперпараметр, или все объекты в узле одного класса
            class_counts = np.bincount(y, minlength=self.num_class)
            predicted_class = np.argmax(class_counts)
            self.tree[node_id] = (self.LEAF_TYPE, predicted_class, class_counts[predicted_class] / y.size)
        else:

            # выбор признака и порога для разделения
            feature_id, threshold = self.__find_threshold(x, y)

            if feature_id is None:
                # Создание листа
                class_counts = np.bincount(y, minlength=self.num_class)
                predicted_class = np.argmax(class_counts)
                self.tree[node_id] = (self.LEAF_TYPE, predicted_class, class_counts[predicted_class] / y.size)
                return

            # разделение данных c помощью выбранного порога и признака
            x_left, x_right, y_left, y_right = self.__div_samples(x, y, feature_id, threshold)
            if x_left.size == 0 or x_right.size == 0:
                # Если не удаётся разделить, делаем текущий узел листом
                class_counts = np.bincount(y, minlength=self.num_class)
                predicted_class = np.argmax(class_counts)
                self.tree[node_id] = (self.LEAF_TYPE, predicted_class, class_counts[predicted_class] / y.size)
                return

            # добавляем новый узел, обновляем важность признаков, далее идем по рекурсии
            self.tree[node_id] = (self.NON_LEAF_TYPE, feature_id, threshold)


            # Вычисляем уменьшение критерия неопределенности для текущего разделения
            impurity_decrease = self.calculate_impurity_decrease(x, y, feature_id, threshold)
            self.feature_importances_[feature_id] += impurity_decrease

            self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
            self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)



    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)

        self.feature_importances_ = np.round(self.feature_importances_ /self.feature_importances_.sum(), 6)

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


    def __predict_proba_node(self, x, node_id, positive_class):
        """
        Предсказывает вероятность в узле
        """
        node = self.tree[node_id]
        if node[0] == self.LEAF_TYPE:
            return np.array([1 - node[2], node[2]]) if node[1] == positive_class else np.array([node[2], 1 - node[2]])
        _, feature_id, threshold = node
        if x[feature_id] <= threshold:
            return self.__predict_proba_node(x, 2 * node_id + 2, positive_class)
        else:
            return self.__predict_proba_node(x, 2 * node_id + 1, positive_class)


    def predict_proba(self, X):
        """
        Возвращает вероятности для каждого объекта, метод нужен для вероятнотностных метрик, типа roc auc
        """
        return np.array([self.__predict_proba_node(x, 0, positive_class=1) for x in X])


    def get_feature_importance(self):
        """
        Выдает важность признаков
        """
        if self.feature_importances_ is None:
            raise ValueError("Модель еще не обучена")

        return self.feature_importances_


    def calculate_impurity_decrease(self, x, y, feature_id, threshold):
        """
        Вычисляет уменьшение критерия неопределенности при разделении по заданному признаку и порогу.
        Используется для поиска самых информативных признаков
        """
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, feature_id, threshold)

        node_impurity = self.calculate_node_impurity(y)

        # можно было бы это притащить из get_impurity, но я не стал, поэтому и написал функцию calculate_node_impurity
        left_impurity = self.calculate_node_impurity(y_left)
        right_impurity = self.calculate_node_impurity(y_right)

        # уменьшение нашего импурити
        impurity_decrease = (node_impurity * x.shape[0] -
                     left_impurity * x_left.shape[0] -
                     right_impurity * x_right.shape[0])

        return impurity_decrease

    def calculate_node_impurity(self, y):
        """
        Вычисляет критерий неопределенности для узла
        """
        class_proba = np.bincount(y) / y.size

        if self.criterion == 'gini':
            impurity = 1 - np.sum(class_proba ** 2)
        elif self.criterion == 'entropy':
            impurity = -np.sum(class_proba * np.log2(class_proba, where = class_proba > 0))
        elif self.criterion == 'misclassification':
            impurity = 1 - np.max(class_proba)
        else:
            raise ValueError("Неверный критерий")

        return impurity

In [ ]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [ ]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target, random_state=52)

In [ ]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [ ]:
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

Совет: Проверьте, что ваша реализация корректно работает с признаками в которых встречаются повторы.
И подумайте, какие еще граничные случаи могут быть.
Например, проверьте, что на таком примере ваша модель корректно работает:

In [ ]:
X = np.array([[1] * 10, [0, 1, 2, 5, 6, 3, 4, 7, 8, 9]]).T
y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
for depth in range(1, 5):
    my_clf = MyDecisionTreeClassifier(max_depth=depth)
    my_clf.fit(X, y)
    print(accuracy_score(y, my_clf.predict(X)))
    print("DEPTH:", depth, "\n\t\tTree:", my_clf.tree, my_clf.predict(X))

## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine.
Для этого используем numpy.

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match).

Данные и описания колонок во вложениях.

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Либо воспользоваться функцией:

In [ ]:
def preprocess_spd_data(df):
    df = df.iloc[:, :97]

    to_drop = [
        'id', 'idg', 'condtn', 'round', 'position', 'positin1', 'order', 'partner',
        'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
        'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o','met_o',
        'field', 'undergra', 'from', 'zipcode', 'income', 'career', 'sports', 'tvsports', 'exercise',
        'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies',
        'concerts', 'music', 'shopping', 'yoga', 'expnum',
        'mn_sat', 'tuition'
    ]

    df = df.drop(to_drop, axis=1)
    df = df.dropna(subset=['age', 'imprelig', 'imprace', 'date'])

    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)

    # attr1 processing
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                                            'amb1_1', 'shar1_1']].sum(axis=1)
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] =\
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T /
     df.loc[:, 'temp_totalsum'].T).T * 100

    # attr2 processing
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                                            'amb2_1', 'shar2_1']].sum(axis=1)
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] =\
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T /
     df.loc[:, 'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)

    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]

        if i != 4:
            feat.remove('shar{}_1'.format(i))

        df = df.drop(feat, axis=1)

    df = df.drop(['wave'], axis=1)
    df = df.dropna()
    return df

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. В качестве метрики можно взять roc-auc.


In [ ]:
df = pd.read_csv('Speed_Dating_Data.csv', encoding='latin1')
df.shape


In [ ]:
preprocessed_df = preprocess_spd_data(df)
preprocessed_df

In [ ]:
y = preprocessed_df.match.to_numpy()
X = preprocessed_df.drop("match", axis=1).to_numpy()

У нас очень несбалансированы классы, поэтому имеет смысл применить SMOTE, это такой метод, который добавляет синтетичесике данные, чтобы выборка стала сбалансированной


In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X, y)

print(X_resampled.shape, y_resampled.shape)

np.unique(y_resampled, return_counts=True)

Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности.
Постройте графики зависимости точности на валидации и трейне от глубины дерева, от минимального числа объектов для сплита. (Т.е должно быть 2 графика, на каждой должны быть 2 кривые - для трейна и валидации)
Какой максимальной точности удалось достигнуть?

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.33, stratify=y_resampled, random_state=52)

In [ ]:
# Посмотрим какие ответы нам выдает решающее дерево из sklearn

sk_tree = DecisionTreeClassifier(min_samples_split=10, max_depth=15,  criterion="gini")
sk_tree.fit(X_train, y_train)

# Тут нам нужна proba, чтобы ro auc нормально работал, так как обычный предикт выдает просто 0 или 1
pred_test_proba = sk_tree.predict_proba(X_test)[:, 1]
pred_test = sk_tree.predict(X_test)

pred_train_proba = sk_tree.predict_proba(X_train)[:, 1]
pred_train = sk_tree.predict(X_train)

# Смотрим метрики
print("TEST")
print(f"Accuracy: {accuracy_score(y_test, pred_test)}")
print(f"ROC AUC: {roc_auc_score(y_test, pred_test_proba)}")

print("TRAIN")
print(f"Accuracy: {accuracy_score(y_train, pred_train)}")
print(f"ROC AUC: {roc_auc_score(y_train, pred_train_proba)}")


Результаты довольно хорошие с такими гиперпараметры, я их взял из графиков ниже

In [ ]:
# Обучаем наше деревео на такиж же гиперпараметрах как дерево из sklearn
my_tree = MyDecisionTreeClassifier(min_samples_split=10, max_depth=15,  criterion="gini")
my_tree.fit(X_train, y_train)


# Тут также как выше
pred_test_proba = my_tree.predict_proba(X_test)[:, 1]
pred_test = my_tree.predict(X_test)

pred_train_proba = my_tree.predict_proba(X_train)[:, 1]
pred_train = my_tree.predict(X_train)

# Смотрим метрики
print("TEST")
print(f"Accuracy: {accuracy_score(y_test, pred_test)}")
print(f"ROC AUC: {roc_auc_score(y_test, pred_test_proba)}")

print("TRAIN")
print(f"Accuracy: {accuracy_score(y_train, pred_train)}")
print(f"ROC AUC: {roc_auc_score(y_train, pred_train_proba)}")


Наше дерево почти такое же как и дерево из sklearn, значит код написан верно

In [ ]:
def get_score_by_depth_or_split(
    X_train, y_train, X_test, y_test,
    DecisionTreeClassifier=DecisionTreeClassifier,
    depth=10, samples_split=10,
    score=accuracy_score,
    kind="depth"):
  """
  Получаем два словаря зависимостей auc-roc или accuracy на трейне и тесте
  от глубины дерева или от минимального числа объектов для сплита
  """

  # словарики для ответов
  res_test = {}
  res_train = {}

  # если идет проверка на глубину
  if kind == "depth":
    for i in range(1, depth):
      # Инициализируем и обучаем дерево
      tree = DecisionTreeClassifier(min_samples_split=samples_split, max_depth=i)
      tree.fit(X_train, y_train)

      # меняем метод в зависимости от метрики, которую мы используем
      if score == accuracy_score:
        pred_test = tree.predict(X_test)
        pred_train = tree.predict(X_train)
      else:
        pred_test = tree.predict_proba(X_test)[:, 1]
        pred_train = tree.predict_proba(X_train)[:, 1]

      res_test[str(i)] = score(y_test, pred_test)
      res_train[str(i)] = score(y_train, pred_train)

  # если идет проверка на минимальное количество объектов в листе
  elif kind == "minsplit":
    for i in range(2, samples_split):
      # Инициализируем и обучаем дерево
      tree = DecisionTreeClassifier(min_samples_split=i, max_depth=depth)
      tree.fit(X_train, y_train)

      # меняем метод в зависимости от метрики, которую мы используем
      if score == accuracy_score:
        pred_test = tree.predict(X_test)
        pred_train = tree.predict(X_train)
      else:
        pred_test = tree.predict_proba(X_test)[:, 1]
        pred_train = tree.predict_proba(X_train)[:, 1]

      res_test[str(i)] = score(y_test, pred_test)
      res_train[str(i)] = score(y_train, pred_train)

  return res_train, res_test

In [ ]:
def plot_auc_roc_by_depth(train_values, test_values, score=accuracy_score, kind="depth"):
  """
  Рисуем график зависимости auc-roc от глубины дерева на трейне и тесте
  """
  plt.figure(figsize=(13, 6))
  plt.plot(range(1, len(test_values) + 1), test_values, label="Test")
  plt.plot(range(1,len(train_values) + 1), train_values, label="Train")

  plt.title(f"Значение {score.__name__} в зависимости от {kind}")
  plt.xlim((1, len(test_values) + 1))
  plt.legend()
  plt.xlabel(f"{kind}")
  plt.ylabel(score.__name__)
  plt.grid()
  plt.show()

In [ ]:
# Получаем значения accuracy для минимального количества объектов от 2 до 49, глубина 15

res_by_depth_train, res_by_depth_test = get_score_by_depth_or_split(X_train, y_train, X_test, y_test,
                                                                        DecisionTreeClassifier=MyDecisionTreeClassifier,
                                                                        depth=15, kind='minsplit',
                                                                        score=accuracy_score,
                                                                        samples_split=50)

Теперь построим график для визуальной оценки зависимости accuracy от минимального количества объектов в листе

In [ ]:
plot_auc_roc_by_depth(res_by_depth_train.values(), res_by_depth_test.values(), kind='minsplit')

In [ ]:
# Получаем значения accuracy для глубины дерева от 1 до 25, минимальное колмчество объектов в листе 10

res_by_depth_train_2, res_by_depth_test_2 = get_score_by_depth_or_split(X_train, y_train, X_test, y_test,
                                                                        DecisionTreeClassifier=MyDecisionTreeClassifier,
                                                                        depth=25, kind='depth',
                                                                        score=accuracy_score,
                                                                        samples_split=10)


Теперь построим график для визуальной оценки зависимости accuracy от глубины в дереве

In [ ]:
plot_auc_roc_by_depth(res_by_depth_train_2.values(), res_by_depth_test_2.values(), kind='depth')

Получилось достичь довольно хорошего качества. На тесте accuracy 0.80, а roc auc 0.86, это при глубине 15 и min_sample_split 10.

Известным фактом является то, что деревья решений сильно переобучаются при увеличении глубины и просто запоминают трейн.
Замечаете ли вы такой эффект судя по графикам? Что при этом происходит с качеством на валидации?

ОТВЕТ: Да, видно на графиках, что деревья переобучаются при большой глубине или маленьком min_sample_split. Чем меньше min_sample_split, тем выше трейн, тест не сильно меняется, но все равно есть небольшие скачки. На сильно больших min_sample_split начинает просаживаться и тест. Чем больше глубина, тем выше трейн, что говорит о переобучении. Тест выходит на плато и сильно не меняется.

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве.

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший.
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева.

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [ ]:
def show_best_features(features):
    """
    Выводит 10 самых важных признаков
    """
    new_pd = pd.DataFrame(
    {"feature": preprocessed_df.drop('match', axis=1).columns, "importance": list(features)}
    )
    return new_pd.sort_values(by="importance", ascending=False).reset_index(drop=True)[:10]

In [ ]:
final_my_tree = MyDecisionTreeClassifier(min_samples_split=10, max_depth=15, criterion="gini")
final_my_tree.fit(X_train, y_train)

In [ ]:
my_features = final_my_tree.get_feature_importance()
my_features

In [ ]:
show_best_features(my_features)

Теперь сравним наши результаты с аналогичным методом из sklearn

In [ ]:
final_sk_tree = DecisionTreeClassifier(min_samples_split=10, max_depth=15, criterion="gini")
final_sk_tree.fit(X_train, y_train)

In [ ]:
sk_features = final_sk_tree.feature_importances_
sk_features

In [ ]:
show_best_features(sk_features)

Мои самые важные признаки совпадают с самыми важными признаками из sklearn, значит код работает верно.

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

Довольно сложно было писать само дерево, очень сложно было реализовывать метод __get_impurity, чтобы он быстро работал. Также довольно много времени потратил на feature_importance, чтобы получить аналогичные результаты как в sklearn. В принципе, нормально, за пару дней делается.

### ВАШ ОТЗЫВ ЗДЕСЬ

